# 1) Import Libraries

In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# 2) Read Dataset

In [226]:
# books = pd.read_csv("dataset - amazon - books.csv",sep=";",error_bad_lines=False, encoding="latin-1", low_memory=False)
# ratings = pd.read_csv("dataset - amazon - ratings.csv",sep=";",error_bad_lines=False, encoding="latin-1", low_memory=False)
# users = pd.read_csv("dataset - amazon - users.csv",sep=";",error_bad_lines=False, encoding="latin-1", low_memory=False)

countries_combine_pivot = pd.read_csv('implementation - presentation - booksPivot.csv', index_col=0)
books3 = pd.read_csv('implementation - presentation - books727.csv')

# ONE TIME RUN

## Users with less than 50 ratings and books less than 100 ratings are excluded. (ONE TIME RUN)

In [204]:
# Users with less than 50 ratings are excluded:
counts1 = ratings['User-ID'].value_counts()
ratings1 = ratings[ratings['User-ID'].isin(counts1[counts1 >= 50].index)]
# Books less than 100 ratings are excluded:
counts2 = ratings['ISBN'].value_counts()
ratings2 = ratings[ratings['ISBN'].isin(counts2[counts2 >= 100].index)]

## Collaborative Filtering Using k-Nearest Neighbors(KNN) (ONE TIME RUN)

In [212]:
# Merge books1 & ratings2
combine_book_rating = pd.merge(ratings2, books1, on='ISBN')
combine_books = combine_book_rating.drop(["Book-Author","Year-Of-Publication","Publisher","Image-URL-L"],axis = 1)

# Added Total_Rating_Count Column
book_ratings_count = (combine_books.groupby(by = ['Book-Title'])['Book-Rating'].count().reset_index().rename(columns = {'Book-Rating' :'Total_Rating_Count'})[['Book-Title','Total_Rating_Count']])
book_ratings_count = book_ratings_count.sort_values(by = 'Total_Rating_Count',ascending=False)

# Merge Total_Rating_Count Column df with Merge of books1 & ratings2 (combine_books)
combine_books1 = combine_books.merge(book_ratings_count,on = 'Book-Title')

# Exclude books having Total_Rating_Count less than 30:
popularity_threshold = 30
combine_books2 = combine_books1.query('Total_Rating_Count >= @popularity_threshold')

# Merge combine_books2 with users df
combine_books3 = combine_books2.merge(users,on='User-ID')

# Include books of only 7 main countries:
countries_combine = combine_books3[combine_books3['Location'].str.contains('usa|canada|united kingdom|germany|spain|australia|italy')]
countries_combine = countries_combine.drop(['Age','Location'],axis = 1)


# Creating Matrix for KNN
countries_combine = countries_combine.drop_duplicates(['User-ID', 'Book-Title'])

# countries_combine_pivot = Book-Title vs User-ID Table
countries_combine_pivot = countries_combine.pivot(index= 'Book-Title',columns = 'User-ID',values = 'Book-Rating').fillna(0)

# Uses Sparse Matrix
countries_combine_matrix = csr_matrix(countries_combine_pivot.values)

# KNN Algorithm
knn_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

# Fit Matrix into KNN Model
knn_model.fit(countries_combine_matrix)

# Export countries_combine_pivot
countries_combine_pivot.to_csv('implementation - presentation - booksPivot.csv', index=True)

## ADD Avg-Rating Column (ONE TIME RUN)

In [206]:
# Find Averages of Books:

# combine_books = [User-ID, ISBN, Book-Rating, Book-Title]
# combine_books11 = exclude all the books which have 0 ratings in total
combine_books11 = combine_books[combine_books['Book-Rating']>=1]
# title_list = list of Book-Title of combine_books11
title_list = list(set(combine_books11['Book-Title'].tolist()))

# avg_rating = [Book-Title, Avg rating]
avg_rating = []
# Find avg book rating of all books and rounding the avg to 1 decimal:
for i in title_list:
    avg_rating.append([i,round(combine_books11[combine_books11['Book-Title']==i]['Book-Rating'].mean(),1)])
    
# Remove NA values from books1
books2 = books1.dropna()
books2.reset_index(inplace=True)


# Add Avg-Rating column to books2
for i in avg_rating:
#     index_list = list of indices which match the Book-Title in books2
    index_list = list(books2[books2['Book-Title']==i[0]].index)
#     Add value to Avg-Rating column of books2 using index_list
    books2.loc[index_list, 'Avg-Rating'] = i[1]
    
    
# books3 = 1915 filtered books
books3 = books2.dropna()
# removed duplicate ISBN books from countries_combine
countries_combine_ISBN_list = list(set(countries_combine.ISBN))
books3 = books3[books3['ISBN'].isin(countries_combine_ISBN_list)]
books3.to_csv('implementation - presentation - books727.csv')

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


# KNN (RUN Everytime)

In [221]:
# Uses Sparse Matrix
countries_combine_matrix = csr_matrix(countries_combine_pivot.values)

# KNN Algorithm
knn_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

# Fit Matrix into KNN Model
knn_model.fit(countries_combine_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

# Find Random Book

In [222]:
# Index of Book-Title from Book-Title vs User-ID Pivot Table
query_index = np.random.choice(countries_combine_pivot.shape[0])
# Index of Book-Title from Book-Title vs User-ID Pivot Table
query_name = countries_combine_pivot.index[query_index]

print(query_index)
print(query_name)

531
The Last Juror


# Final Output

In [223]:
# countries_combine_pivot = Book-Title vs User-ID Table
# .iloc -> Display row of query_index with User-ID
# .values -> convert the .iloc to values
# .reshape(1,-1) -> Convert the single array into array of array
# .reshape(1,-1) -> .reshape(rows, columns) {1 value indicates 1 row, -1 indicates no proper value for columns}
countries_combine_reshape = countries_combine_pivot.iloc[query_index,:].values.reshape(1,-1)
# distances and indices of output:
distances, indices = knn_model.kneighbors(countries_combine_reshape, n_neighbors= 11)

recommend_collab_names = []
for i in indices[0][1:]:
    recommend_collab_names.append(countries_combine_pivot.index[i])
    
recommend_collab = books3[books3['Book-Title'].isin(recommend_collab_names)].groupby('Book-Title').first()
recommend_collab.reset_index(inplace=True)

In [224]:
recommend_collab_names

['Blow Fly: A Scarpetta Novel',
 'Four Blind Mice',
 'The King of Torts',
 'The Big Bad Wolf: A Novel',
 'The Jester',
 'Key of Valor (Roberts, Nora. Key Trilogy, 3.)',
 'Bleachers',
 'The Five People You Meet in Heaven',
 'Q Is for Quarry',
 'Mortal Prey']

In [225]:
recommend_collab

,Book-Title,Unnamed: 0,index,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Avg-Rating
0,Bleachers,221,221,0385511612,John Grisham,2003,Doubleday,http://images.amazon.com/images/P/0385511612.0...,6.6
1,Blow Fly: A Scarpetta Novel,4991,4991,0399150897,Patricia Cornwell,2003,Putnam Publishing Group,http://images.amazon.com/images/P/0399150897.0...,7.3
2,Four Blind Mice,205,205,0316693006,James Patterson,2002,"Little, Brown",http://images.amazon.com/images/P/0316693006.0...,8.2
3,"Key of Valor (Roberts, Nora. Key Trilogy, 3.)",16198,16198,0515136530,Nora Roberts,2003,Jove Pubns,http://images.amazon.com/images/P/0515136530.0...,8.5
4,Mortal Prey,22051,22051,0425189864,John Sandford,2003,Berkley Publishing Group,http://images.amazon.com/images/P/0425189864.0...,7.9
5,Q Is for Quarry,7364,7364,0425192725,Sue Grafton,2003,Berkley Publishing Group,http://images.amazon.com/images/P/0425192725.0...,7.9
6,The Big Bad Wolf: A Novel,204,204,0316602906,James Patterson,2003,"Little, Brown",http://images.amazon.com/images/P/0316602906.0...,7.8
7,The Five People You Meet in Heaven,107,107,0786868716,Mitch Albom,2003,Hyperion,http://images.amazon.com/images/P/0786868716.0...,8.0
8,The Jester,2419,2419,0316602051,James Patterson,2003,"Little, Brown",http://images.amazon.com/images/P/0316602051.0...,7.6
9,The King of Torts,219,219,0385508042,John Grisham,2003,Doubleday Books,http://images.amazon.com/images/P/0385508042.0...,7.5


In [57]:
def getImageURL(input1):
    return books2[books2['ISBN']==input1]['Image-URL-L']

In [58]:
getImageURL()

TypeError: getImageURL() missing 1 required positional argument: 'input1'